In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import models,transforms,datasets
from torch.utils import data
import os

In [2]:
# boxes1 = torch.tensor(
#     [
#         [1, 1, 3, 2],
#         [1.5, 1, 2.5, 2],
#         [1, 1.5, 3, 2.5]
#     ]
# )
# boxes2 = torch.tensor(
#     [
#         [2, 1.5, 3, 3],
#         [0, 1, 2, 3]
#     ]
# )
# inter_upperlefts = torch.maximum(boxes1[:, None, :2], boxes2[:, :2])
# print("inter_upperlefts\n", inter_upperlefts)
# print("inter_upperlefts.shape\n", inter_upperlefts.shape)
# inter_lowerrights = torch.minimum(boxes1[:, None, 2:], boxes2[:, 2:])
# print("inter_lowerrights\n", inter_lowerrights)
# print("inter_lowerrights.shape\n", inter_lowerrights.shape)

In [3]:

def box_iou(boxes1, boxes2):
    """
    输入：boxes1:[boxes1数量,(左上x,左上y,右下x,右下y)],boxes2:[boxes2数量,4]，boxes1是锚框，boxes2是真实gt边界框
    输出：交并比[boxes1数量,boxes2数量]
    """
    #定义lambda表达式，计算矩形框的面积
    box_area = lambda boxes: ((boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1]))

    #计算锚框（anchor box）的面积,[boxes1数量]
    areas1 = box_area(boxes1)
    #计算真实边界框gt bounding box的面积,[boxes2数量]
    areas2 = box_area(boxes2)

    # 相交区域inter_upperlefts, inter_lowerrights, inters的形状:
    # (boxes1的数量,boxes2的数量,2)
    inter_upperlefts = torch.maximum(boxes1[:, None, :2], boxes2[:, :2])
    # None 的作用是在第二个维度插入一个新的轴，即将数组变为 (boxes1的数量, 1, 4) 的形状。
    inter_lowerrights = torch.minimum(boxes1[:, None, 2:], boxes2[:, 2:])
    # 得到交集的宽和高，[boxes1数量,boxes2数量,2],clamp限制输出最小为0(min=0)
    inters = (inter_lowerrights - inter_upperlefts).clamp(min=0)
    # 得到交集的面积[boxes1数量,boxes2数量]
    inter_areas = inters[:, :, 0] * inters[:, :, 1]
    # 并集的面积等于两个边界框列表中所有边界框的面积之和减去交集的面积。
    # 得到并集的面积[boxes1数量,boxes2数量],此处又用到了广播机制
    union_areas = areas1[:, None] + areas2 - inter_areas
    return inter_areas / union_areas

def assign_anchor_to_bbox(ground_truth, anchors, device, iou_threshold=0.5):
    """将最接近的真实边界框分配给锚框"""
    num_anchors, num_gt_boxes = anchors.shape[0], ground_truth.shape[0]
    # 位于第i行和第j列的元素x_ij是锚框i和真实边界框j的IoU
    jaccard = box_iou(anchors, ground_truth)
    # 对于每个锚框，分配的真实边界框的张量
    anchors_bbox_map = torch.full((num_anchors,), -1, dtype=torch.long, device=device)
    # 遍历交并比矩阵每一行，寻找每一行最大值，根据阈值，决定是否分配真实边界框
    max_ious, indices = torch.max(jaccard, dim=1)
    anc_i = torch.nonzero(max_ious >= iou_threshold).reshape(-1)
    box_j = indices[max_ious >= iou_threshold]
    anchors_bbox_map[anc_i] = box_j
    # 在jaccard交并比矩阵中，寻找每个真实框的最大值
    col_discard = torch.full((num_anchors,), -1)
    row_discard = torch.full((num_gt_boxes,), -1)
    for _ in range(num_gt_boxes):
        max_idx = torch.argmax(jaccard)
        box_idx = (max_idx % num_gt_boxes).long()
        anc_idx = (max_idx / num_gt_boxes).long()
        anchors_bbox_map[anc_idx] = box_idx
        jaccard[:, box_idx] = col_discard
        jaccard[anc_idx, :] = row_discard
    return anchors_bbox_map

def box_corner_to_center(boxes):
    """从（左上，右下）转换到（中间，宽度，高度）"""
    x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    w = x2 - x1
    h = y2 - y1
    boxes = torch.stack((cx, cy, w, h), axis=-1)
    return boxes

def offset_boxes(anchors, assigned_bb, eps=1e-6):
    """对锚框偏移量的转换"""
    c_anc = box_corner_to_center(anchors)
    c_assigned_bb = box_corner_to_center(assigned_bb)
    offset_xy = 10 * (c_assigned_bb[:, :2] - c_anc[:, :2]) / c_anc[:, 2:]
    offset_wh = 5 * torch.log(eps + c_assigned_bb[:, 2:] / c_anc[:, 2:])
    offset = torch.cat([offset_xy, offset_wh], axis=1)
    return offset

def multibox_target(anchors, labels):
    """使用真实边界框标记锚框"""
    # .squeeze(0) 方法将删除第一个维度为 1 的维度。这种操作通常用于减少张量的尺寸，
    # 如果第一个维度确实是长度为 1 的维度，那么它将被压缩。如果第一个维度不是长度为 1 的维度，则不会进行任何操作。
    batch_size, anchors = labels.shape[0], anchors.squeeze(0)
    batch_offset, batch_mask, batch_class_labels = [], [], []
    device, num_anchors = anchors.device, anchors.shape[0]
    for i in range(batch_size):
        # labels[batch_size, num_gt_boxes, [?, 左上x, 左上y, 右下x, 右下y]]
        label = labels[i, :, :]
        # label[num_gt_boxes, [?, 左上x, 左上y, 右下x, 右下y]]
        anchors_bbox_map = assign_anchor_to_bbox(label[:, 1:], anchors, device)
        # anchors_bbox_map: [], size = num_anchors = anchors.shape[0]
        # 扩展为四列
        bbox_mask = ((anchors_bbox_map >= 0).float().unsqueeze(-1)).repeat(1, 4)
        # 将类标签和分配的边界框坐标初始化为零
        class_labels = torch.zeros(num_anchors, dtype=torch.long, device=device)
        # class_labels.shape : [num_anchors]
        assigned_bb = torch.zeros((num_anchors, 4), dtype=torch.float32, device=device)
        # assigned_bb.shape : [num_anchors, 4]

        # 使用真实边界框来标记锚框的类别。
        # 如果一个锚框没有被分配，标记其为背景（值为零）
        indices_true = torch.nonzero(anchors_bbox_map >= 0)
        bb_idx = anchors_bbox_map[indices_true]
        
        class_labels[indices_true] = label[bb_idx, 0].long() + 1
        assigned_bb[indices_true] = label[bb_idx, 1:]
        # 偏移量转换
        offset = offset_boxes(anchors, assigned_bb) * bbox_mask
        batch_offset.append(offset.reshape(-1))
        batch_mask.append(bbox_mask.reshape(-1))
        batch_class_labels.append(class_labels)
    bbox_offset = torch.stack(batch_offset)
    bbox_mask = torch.stack(batch_mask)
    class_labels = torch.stack(batch_class_labels)
    return (bbox_offset, bbox_mask, class_labels)


In [31]:
def box_center_to_corner(boxes):
    """从（中间，宽度，高度）转换到（左上，右下）"""
    cx, cy, w, h = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]
    x1 = cx - 0.5 * w
    y1 = cy - 0.5 * h
    x2 = cx + 0.5 * w
    y2 = cy + 0.5 * h
    boxes = torch.stack((x1, y1, x2, y2), axis=-1)
    return boxes

def offset_inverse(anchors, offset_preds):
    """根据带有预测偏移量的锚框来预测边界框"""
    # 从（左上，右下）转换到（中间，宽度，高度）
    anc = box_corner_to_center(anchors)
    pred_bbox_xy = (offset_preds[:, :2] * anc[:, 2:] / 10) + anc[:, :2]
    pred_bbox_wh = torch.exp(offset_preds[:, 2:] / 5) * anc[:, 2:]
    pred_bbox = torch.cat((pred_bbox_xy, pred_bbox_wh), axis=1)
    predicted_bbox = box_center_to_corner(pred_bbox)
    return predicted_bbox

def nms(boxes, scores, iou_threshold):
    """
    非极大值抑制（non-maximum suppression，NMS），对预测边界框的置信度进行排序
    ，保留预测边界框的指标.
    输出：keep 是索引值
    """
    # 按照最后一个维度进行降序排序, 返回的是索引值
    B = torch.argsort(scores, dim=-1, descending=True)
    keep = []  # 保留预测边界框的指标
    while B.numel() > 0:
        # 从B中选取置信度最高的预测边界框B[0]作为基准
        i = B[0]
        keep.append(i)
        # 如果只剩最后一个元素
        if B.numel() == 1: break
        iou = box_iou(
            boxes[i, :].reshape(-1, 4), 
            boxes[B[1:], :].reshape(-1, 4)
        ).reshape(-1)
        #  函数用于找到所有为 True 的元素的索引
        inds = torch.nonzero(iou <= iou_threshold).reshape(-1)
        B = B[inds + 1]
    return torch.tensor(keep, device=boxes.device)

def multibox_detection(cls_probs, offset_preds, anchors, nms_threshold=0.5,
                       pos_threshold=0.009999999):
    """使用非极大值抑制来预测边界框"""
    device, batch_size = cls_probs.device, cls_probs.shape[0]
    anchors = anchors.squeeze(0)
    # num_classes = cls_probs.shape[1]
    num_anchors = cls_probs.shape[2]
    out = []
    for i in range(batch_size):
        # 第i个batch_size
        cls_probs, offset_pred = cls_probs[i], offset_preds[i].reshape(-1, 4)
        print(1, "cls_probs[1:]\n", cls_probs[1:])
        conf, class_id = torch.max(cls_probs[1:], 0)
        print(2, "conf\n", conf)
        print(2, "class_id\n", class_id)
        predicted_bb = offset_inverse(anchors, offset_pred)
        print(3, "predicted_bb\n", predicted_bb)
        keep = nms(predicted_bb, conf, nms_threshold)
        print(4, "keep\n", keep)

        # 找到所有的non_keep索引， 并将类设置为背景
        all_idx = torch.arange(num_anchors, dtype=torch.long, device=device)
        print(5, "all_idx\n", all_idx)
        combined = torch.cat((keep, all_idx))
        print(6, "combined\n", combined)
        print(7, "combined.unique(return_counts=True)\n", combined.unique(return_counts=True))
        uniques, counts = combined.unique(return_counts=True)
        print(7, "uniques\n", uniques)
        print(7, "counts\n", counts)
        non_keep = uniques[counts == 1]
        print(8, "non_keep\n", non_keep)
        all_id_sorted = torch.cat((keep, non_keep))
        print(9, "all_id_sorted\n", all_id_sorted)

        class_id[non_keep] = -1
        print(10, "class_id\n", class_id)
        class_id = class_id[all_id_sorted]
        print(10.1, "class_id\n", class_id)
        conf, predicted_bb = conf[all_id_sorted], predicted_bb[all_id_sorted]
        print(11, "conf\n", conf)
        print(11.1, "predicted_bb\n", predicted_bb)
        # pos_threshold是一个用于非常背景预测的阈值
        below_min_idx = (conf < pos_threshold)
        print(12, "below_min_idx\n", below_min_idx)
        conf[below_min_idx] = 1.0 - conf[below_min_idx]

        print(13, "conf\n", conf)
        pred_info = torch.cat(
            (class_id.unsqueeze(1), conf.unsqueeze(1), predicted_bb),
            dim=1
        )
        print(13, "pred_info\n", pred_info.shape)
        out.append(pred_info)
        print(14, "torch.stack(out)\n", torch.stack(out))
        return torch.stack(out)

In [32]:
anchors = torch.tensor(
    [
        [0.1, 0.08, 0.52, 0.92],
        [0.08, 0.2, 0.56, 0.95],
        [0.15, 0.3, 0.62, 0.91],
        [0.55, 0.2, 0.9, 0.88]
    ]
)
offset_preds = torch.tensor([0] * anchors.numel())
cls_probs = torch.tensor(
    [
        [0] * 4,  # 背景的预测概率
        [0.9, 0.8, 0.7, 0.1],  # 狗的预测概率
        [0.1, 0.2, 0.3, 0.9]
    ]
)  # 猫的预测概率

output = multibox_detection(cls_probs.unsqueeze(dim=0),
                            offset_preds.unsqueeze(dim=0),
                            anchors.unsqueeze(dim=0),
                            nms_threshold=0.5)

1 cls_probs[1:]
 tensor([[0.9000, 0.8000, 0.7000, 0.1000],
        [0.1000, 0.2000, 0.3000, 0.9000]])
2 conf
 tensor([0.9000, 0.8000, 0.7000, 0.9000])
2 class_id
 tensor([0, 0, 0, 1])
3 predicted_bb
 tensor([[0.1000, 0.0800, 0.5200, 0.9200],
        [0.0800, 0.2000, 0.5600, 0.9500],
        [0.1500, 0.3000, 0.6200, 0.9100],
        [0.5500, 0.2000, 0.9000, 0.8800]])
4 keep
 tensor([0, 3])
5 all_idx
 tensor([0, 1, 2, 3])
6 combined
 tensor([0, 3, 0, 1, 2, 3])
7 combined.unique(return_counts=True)
 (tensor([0, 1, 2, 3]), tensor([2, 1, 1, 2]))
7 uniques
 tensor([0, 1, 2, 3])
7 counts
 tensor([2, 1, 1, 2])
8 non_keep
 tensor([1, 2])
9 all_id_sorted
 tensor([0, 3, 1, 2])
10 class_id
 tensor([ 0, -1, -1,  1])
10.1 class_id
 tensor([ 0,  1, -1, -1])
11 conf
 tensor([0.9000, 0.9000, 0.8000, 0.7000])
11.1 predicted_bb
 tensor([[0.1000, 0.0800, 0.5200, 0.9200],
        [0.5500, 0.2000, 0.9000, 0.8800],
        [0.0800, 0.2000, 0.5600, 0.9500],
        [0.1500, 0.3000, 0.6200, 0.9100]])
12 below_